In [19]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from keras.layers import MaxPooling2D
from keras.layers import Dropout, Activation, Flatten
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
from sklearn.metrics import confusion_matrix
import numpy as np
import pickle
from sklearn.utils import shuffle
import keras.backend as K
import keras.utils

img_size = 32 # Width and height of each image.
num_channels = 3 # Number of channels in each image, 3 channels: Red, Green, Blue.
img_size_flat = img_size * img_size * num_channels # Length of an image when flattened to a 1-dim array.
num_classes = 10 # Number of classes.
_num_files_train = 5 # Number of files for the training-set.
_images_per_file = 10000 # Number of images for each batch-file in the training-set.
_num_images_train = _num_files_train * _images_per_file # Total number of images in the training-set. This is used to pre-allocate arrays for efficiency.

In [2]:
def createModel(input_shape):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same',
                     input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
     
    return model

In [20]:
def load_batch(fpath, label_key='labels'):
    """Internal utility for parsing CIFAR data.
    # Arguments
        fpath: path the file to parse.
        label_key: key for label data in the retrieve
            dictionary.
    # Returns
        A tuple `(data, labels)`.
    """
    
    with open(fpath, 'rb') as f:
        d = pickle.load(f, encoding='bytes')
        # decode utf8
        d_decoded = {}
        for k, v in d.items():
            d_decoded[k.decode('utf8')] = v
        d = d_decoded
    data = d['data']
    labels = d[label_key]

    data = data.reshape(data.shape[0], 3, 32, 32)
    return data, labels

def load_data():
    num_train_samples = 50000

    x_train = np.empty((num_train_samples, 3, 32, 32), dtype='uint8')
    y_train = np.empty((num_train_samples,), dtype='uint8')

    for i in range(1, 6):
        fpath = 'cifar10/data_batch_' + str(i)
        (x_train[(i - 1) * 10000: i * 10000, :, :, :],
         y_train[(i - 1) * 10000: i * 10000]) = load_batch(fpath)

    fpath = 'cifar10/test_batch'
    x_test, y_test = load_batch(fpath)

    y_train = np.reshape(y_train, (len(y_train), 1))
    y_test = np.reshape(y_test, (len(y_test), 1))
    
    x_train = x_train.transpose(0, 2, 3, 1)
    x_test = x_test.transpose(0, 2, 3, 1)
    
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)

    return x_train, y_train, x_test, y_test

In [21]:
x_train, y_train, x_test, y_test = load_data()
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(50000, 32, 32, 3)
(50000, 10)
(10000, 32, 32, 3)
(10000, 10)


In [23]:
model = createModel(x_train.shape[1:])
batch_size = 32
epochs = 100

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False) # randomly flip images

datagen.fit(x_train)

# fits the model on batches with real-time data augmentation:
model.fit_generator(datagen.flow(x_train, y_train,
                                 batch_size=batch_size),
                                 epochs=epochs,
                                 validation_data=(x_test, y_test),
                                 workers=4)

Epoch 1/100
1563/1563 [==============================] - 34s 22ms/step - loss: 1.8682 - acc: 0.3146 - val_loss: 1.5600 - val_acc: 0.4350
Epoch 2/100
1563/1563 [==============================] - 30s 19ms/step - loss: 1.5946 - acc: 0.4159 - val_loss: 1.6081 - val_acc: 0.4331
Epoch 3/100
1563/1563 [==============================] - 32s 21ms/step - loss: 1.4669 - acc: 0.4671 - val_loss: 1.3112 - val_acc: 0.5255
Epoch 4/100
1563/1563 [==============================] - 29s 19ms/step - loss: 1.3798 - acc: 0.5047 - val_loss: 1.2210 - val_acc: 0.5636
Epoch 5/100
1563/1563 [==============================] - 28s 18ms/step - loss: 1.3030 - acc: 0.5346 - val_loss: 1.1590 - val_acc: 0.5898
Epoch 6/100
1563/1563 [==============================] - 30s 19ms/step - loss: 1.2481 - acc: 0.5571 - val_loss: 1.0720 - val_acc: 0.6249
Epoch 7/100
1563/1563 [==============================] - 32s 21ms/step - loss: 1.1886 - acc: 0.5768 - val_loss: 1.0324 - val_acc: 0.6399
Epoch 8/100
1563/1563 [==================

KeyboardInterrupt: 